<a href="https://colab.research.google.com/github/trkfz2/M2020/blob/master/NLP_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Load and prepare data**

## 1. Load the dataset into a pandas dataframe.

In [0]:
import nltk
import pandas as pd
import re

In [2]:
!wget https://alexip-ml.s3.amazonaws.com/stackexchange_812k.csv.gz

--2020-03-02 12:17:31--  https://alexip-ml.s3.amazonaws.com/stackexchange_812k.csv.gz
Resolving alexip-ml.s3.amazonaws.com (alexip-ml.s3.amazonaws.com)... 52.216.98.187
Connecting to alexip-ml.s3.amazonaws.com (alexip-ml.s3.amazonaws.com)|52.216.98.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130230720 (124M) [application/x-gzip]
Saving to: ‘stackexchange_812k.csv.gz’

stackexchange_812k. 100%[===================>] 124.20M  33.3MB/s    in 3.9s    

2020-03-02 12:17:36 (31.8 MB/s) - ‘stackexchange_812k.csv.gz’ saved [130230720/130230720]



In [0]:
!gunzip stackexchange_812k.csv.gz

In [0]:
df = pd.read_csv("stackexchange_812k.csv")

In [5]:
df.shape

(812132, 5)

In [6]:
df.columns

Index(['post_id', 'parent_id', 'comment_id', 'text', 'category'], dtype='object')

In [7]:
df.dtypes

post_id         int64
parent_id     float64
comment_id    float64
text           object
category       object
dtype: object

## 2. Use regular expressions to remove elements that are not words
 such as:
 - html tags, 
 - latex expressions, 
 - urls, 
 - digits, 
 - line returns, …


In [0]:
def remove_html_tags(text):
    """Remove html tags from a string"""  
    clean = re.compile('<.*?>') 
    return re.sub(clean, '', text)

In [0]:
def remove_latex(text):
    """Remove html tags from a string"""  
    clean = re.compile('\$.*?\$') 
    return re.sub(clean, '', text)


In [0]:
def remove_url(text):
    """"Remove urls from a string"""  
    text = re.sub(r'(http\S+|ftp\S+|www\S+)', '', text) 
    return(text)

In [0]:
def remove_specialchar(text):
    """"Remove urls from a string"""  
    text = re.sub('[^A-Za-z,.!? ]+', '', text) #'[^A-Za-z0-9 .!,:?]+'
    return(text)

 Remove:

In [0]:
# remove html
df['text'] = df['text'].apply(lambda x: remove_html_tags(x))

In [0]:
# remove latex
df['text'] = df['text'].apply(lambda x: remove_latex(x))

In [0]:
# remove url
df['text'] = df['text'].apply(lambda x: remove_url(x))

In [0]:
# remove special characters 
df['text'] = df['text'].apply(lambda x: remove_specialchar(x))

## 3. Remove missing values for texts

In [0]:
import numpy as np

In [0]:
na = df['text'].isna()

In [19]:
sum(na == False) == len(df['text'])

True

## 4. Remove texts that are extremely large or too short
 to bring any information to the model. We want to keep paragraphs that contain at least a few words and remove the paragraphs that are composed of large numerical tables.

In [0]:
text_length = df['text'].apply(lambda x: len(x))

In [23]:
min(text_length), max(text_length)

(0, 21933)

Short text (number of characters < 30):

In [0]:
short_text = df[text_length < 30]

In [25]:
short_text.shape

(27934, 5)

Show some examples:

In [26]:
df['category'].unique()

array(['title', 'post', 'comment'], dtype=object)

In [27]:
short_text[short_text.category == 'post'][0:10]

,post_id,parent_id,comment_id,text,category
91896,259,258.0,NaN,Support vector machine,post
91927,326,114.0,NaN,XIANS OG,post
91985,435,423.0,NaN,One more Dilbert cartoon...,post
92036,546,423.0,NaN,Another one from xkcd,post
92105,656,423.0,NaN,,post
92746,1890,1883.0,NaN,The creation of this site,post
92761,1920,1904.0,NaN,ACM SIGKDD KDD in San Diego,post
92970,2293,1906.0,NaN,NIPS,post
93046,2431,2423.0,NaN,All of Statistics,post
93210,2729,2728.0,NaN,What about silhouette?,post


In [28]:
short_text[short_text.category == 'comment'][0:10]

,post_id,parent_id,comment_id,text,category
259062,4,NaN,15949.0,A related question,comment
259076,7,NaN,800725.0,,comment
259082,9,NaN,2723.0,James,comment
259099,18,NaN,7.0,also the US census data,comment
259110,26,NaN,127496.0,Here is a good explanation,comment
259121,30,NaN,245.0,Similar question on SO,comment
259133,36,NaN,18.0,,comment
259149,43,NaN,206.0,"fine, you win",comment
259151,44,NaN,362884.0,is this your homework?,comment
259205,75,NaN,85.0,,comment


In [29]:
short_text[short_text.category == 'title'][0:10]

,post_id,parent_id,comment_id,text,category
0,1,NaN,NaN,Eliciting priors from experts,title
1,2,NaN,NaN,What is normality?,title
11,23,NaN,NaN,Finding the PDF given the CDF,title
13,26,NaN,NaN,What is a standard deviation?,title
19,44,NaN,NaN,Explain data visualization,title
37,138,NaN,NaN,Free resources for learning R,title
38,145,NaN,NaN,Free Dataset Resources?,title
42,170,NaN,NaN,Free statistical textbooks,title
57,249,NaN,NaN,Variance components,title
63,290,NaN,NaN,Resources for learning Stata,title


Remove short text

In [0]:
df = df[~df.index.isin(short_text.index)]

In [31]:
df.shape

(784198, 5)

In [32]:
new_length = df['text'].apply(lambda x: len(x))
min(new_length), max(new_length)

(30, 21933)

Long text (number of characters > 10000):

In [0]:
long_text = df[text_length > 10000]

In [34]:
long_text.shape

(143, 5)

Show some examples

In [0]:
pd.options.display.max_colwidth = max(text_length)

In [36]:
long_text[long_text.category == 'post'][0:2]

post_id  ...  category
92968     2287  ...      post
93715     3673  ...      post

[2 rows x 5 columns]

In [0]:
pd.options.display.max_colwidth = 50

In [0]:
df = df[~df.index.isin(long_text.index)]

In [39]:
df.shape

(784055, 5)

In [40]:
df.iloc[0], df.iloc[45457], df.iloc[-1]

(post_id                                                       3
 parent_id                                                   NaN
 comment_id                                                  NaN
 text          What are some valuable Statistical Analysis op...
 category                                                  title
 Name: 2, dtype: object,
 post_id                                                  288819
 parent_id                                                   NaN
 comment_id                                                  NaN
 text          Using categorical variable in Conditional Logi...
 category                                                  title
 Name: 49662, dtype: object,
 post_id                                                  279999
 parent_id                                                   NaN
 comment_id                                               542550
 text          As per your other question, your data does not...
 category                           

In [41]:
new_length = df['text'].apply(lambda x: len(x))
min(new_length), max(new_length)

(30, 9998)

## 5. Use a tokenizer
 to create a version of the original text that is a string of space-separated lowercase tokens. 


In [0]:
from nltk.tokenize import WordPunctTokenizer 
     
# Create a reference variable for Class WordPunctTokenizer 
tk = WordPunctTokenizer() 

df['text'] = df['text'].apply(lambda x: tk.tokenize(x)) 
     


In [43]:
print(df.iloc[0])
print('---')
print(df.iloc[456785])

post_id                                                       3
parent_id                                                   NaN
comment_id                                                  NaN
text          [What, are, some, valuable, Statistical, Analy...
category                                                  title
Name: 2, dtype: object
---
post_id                                                  362481
parent_id                                                   NaN
comment_id                                               680911
text          [This, is, possibly, better, asked, on, crossv...
category                                                comment
Name: 472449, dtype: object


## 6. Export the resulting dataframe into a csv file

In [0]:
df.to_csv('tokenized_text.csv', index = True)